In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import numpy as np
import deepchem as dc
from deepchem.molnet import load_delaney
from deepchem.models import GraphConvModel
import tensorflow as tf

/Users/KHatti001/opt/anaconda3/envs/chem_ml/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/KHatti001/opt/anaconda3/envs/chem_ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/KHatti001/opt/anaconda3/envs/chem_ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depreca

#Setting up model

In [2]:
##input malaria file
delaney_tasks = ['Desired_property']
featurizer = dc.feat.ConvMolFeaturizer()

input_dataset = "/Users/KHatti001/OneDrive - University of Dundee/project1_data/Series4_firstSet_trimmed_subset.csv"

loader = dc.data.CSVLoader(tasks=delaney_tasks, smiles_field="smiles", featurizer=featurizer)
dataset = loader.featurize(input_dataset, shard_size=8192)

# Initialize transformers
transformers = [
  dc.trans.NormalizationTransformer(
      transform_y=True, dataset=dataset)
]

print("About to transform data")
for transformer in transformers:
    dataset = transformer.transform(dataset)

# splitters = {
#   'index': dc.splits.IndexSplitter(),
#   'random': dc.splits.RandomSplitter(),
#   'scaffold': dc.splits.ScaffoldSplitter()
# }
splitter = dc.splits.ScaffoldSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)


Loading raw samples now.
shard_size: 8192
About to start loading CSV from /Users/KHatti001/OneDrive - University of Dundee/project1_data/Series4_firstSet_trimmed_subset.csv
Loading shard 1 of size 8192.
Featurizing sample 0
TIMING: featurizing shard 0 took 0.136 s
TIMING: dataset construction took 0.225 s
Loading dataset from disk.
About to transform data
TIMING: dataset construction took 0.032 s
Loading dataset from disk.
TIMING: dataset construction took 0.028 s
Loading dataset from disk.
TIMING: dataset construction took 0.019 s
Loading dataset from disk.
TIMING: dataset construction took 0.015 s
Loading dataset from disk.


In [82]:
# Load Delaney dataset
# delaney_tasks, delaney_datasets, transformers = load_delaney(
#     featurizer='GraphConv', split='index')
# train_dataset, valid_dataset, test_dataset = dataset

# Fit models
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)

# Do setup required for tf/keras models
# Number of features on conv-mols
n_feat = 70
# Batch size of models
batch_size = 256
model = GraphConvModel(
    len(delaney_tasks), batch_size=batch_size, mode='regression')


In [38]:
print("training data length", len(train_dataset))
print("validation data length", len(valid_dataset))

training data length 71
validation data length 9


##Train model

In [83]:
# Fit trained model
def run_and_store_results(nb_epoch_value):
   
    model.fit(train_dataset, nb_epoch=nb_epoch_value)

#     print("Evaluating model")
    train_scores =  model.evaluate(train_dataset, [metric], transformers)
    valid_scores = model.evaluate(valid_dataset, [metric], transformers)
    print("printing some value", train_scores["mean-pearson_r2_score"])
    return train_scores["mean-pearson_r2_score"],valid_scores["mean-pearson_r2_score"]

#     print("Train scores")
#     print(train_scores)

#     print("Validation scores")
#     print(valid_scores)

# model.restore()

In [84]:
valid_scores_list=[]
train_scores_list=[]
epoch_inputs_list=[]
for nb_epoch_value in range(0,50,5):
    print (nb_epoch_value)
    train_scores,valid_scores = run_and_store_results(nb_epoch_value)
    epoch_inputs_list.append(nb_epoch_value)
    valid_scores_list.append(valid_scores)
    train_scores_list.append(train_scores)
    
print(epoch_inputs_list)
print(valid_scores_list)
print(train_scores_list)
    
# dir(train_scores)
# print(dic["C"])
# print(valid_scores["mean-pearson_r2_score"])

0
computed_metrics: [0.09138322797533145]
computed_metrics: [0.033038860831720473]
printing some value 0.09138322797533145
5


/Users/KHatti001/opt/anaconda3/envs/chem_ml/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/Users/KHatti001/opt/anaconda3/envs/chem_ml/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/Users/KHatti001/opt/anaconda3/envs/chem_ml/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


computed_metrics: [0.24156467625858424]
computed_metrics: [0.5488038911576345]
printing some value 0.24156467625858424
10
computed_metrics: [0.2738360809932986]
computed_metrics: [0.2860634291287039]
printing some value 0.2738360809932986
15
computed_metrics: [0.473468570375819]
computed_metrics: [0.3255884424542823]
printing some value 0.473468570375819
20
computed_metrics: [0.6054597981500817]
computed_metrics: [0.3551060362219851]
printing some value 0.6054597981500817
25
computed_metrics: [0.705675333943457]
computed_metrics: [0.32191222528970737]
printing some value 0.705675333943457
30
computed_metrics: [0.7695438446261077]
computed_metrics: [0.20485436510245994]
printing some value 0.7695438446261077
35
computed_metrics: [0.7984239771202608]
computed_metrics: [0.10815385334143347]
printing some value 0.7984239771202608
40
computed_metrics: [0.8106917587657612]
computed_metrics: [0.06463442757487856]
printing some value 0.8106917587657612
45
computed_metrics: [0.8140642486221776]

In [ ]:
model.save()
model = model.load_from_dir('models')
valid_scores = model.evaluate(valid_dataset, [metric], transformers)

In [50]:
jinga=[]
jinga.append(10)

In [51]:
jinga

[10]